In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf MP_DATA

In [ ]:
!unzip /content/drive/MyDrive/MP_DATA.zip

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: MP_DATA/y/44/28.npy     
  inflating: MP_DATA/y/44/29.npy     
  inflating: MP_DATA/y/44/3.npy      
  inflating: MP_DATA/y/44/4.npy      
  inflating: MP_DATA/y/44/5.npy      
  inflating: MP_DATA/y/44/6.npy      
  inflating: MP_DATA/y/44/7.npy      
  inflating: MP_DATA/y/44/8.npy      
  inflating: MP_DATA/y/44/9.npy      
   creating: MP_DATA/y/45/
  inflating: MP_DATA/y/45/0.npy      
  inflating: MP_DATA/y/45/1.npy      
  inflating: MP_DATA/y/45/10.npy     
  inflating: MP_DATA/y/45/11.npy     
  inflating: MP_DATA/y/45/12.npy     
  inflating: MP_DATA/y/45/13.npy     
  inflating: MP_DATA/y/45/14.npy     
  inflating: MP_DATA/y/45/15.npy     
  inflating: MP_DATA/y/45/16.npy     
  inflating: MP_DATA/y/45/17.npy     
  inflating: MP_DATA/y/45/18.npy     
  inflating: MP_DATA/y/45/19.npy     
  inflating: MP_DATA/y/45/2.npy      
  inflating: MP_DATA/y/45/20.npy     
  inflating: MP_DATA/y/45/21.npy  

In [ ]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install mediapipe matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [ ]:
import cv2
import numpy as np
import os
import datetime
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
DATA_PATH = os.path.join('MP_DATA')
actions = np.array(open('/content/drive/MyDrive/classes.txt', 'r').read().split('\n'))
number_of_sequences = 100
every_sequence_length = 30
sequences, labels = [], []

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'w': 20,
 'y': 21,
 'z': 22}

In [ ]:
for action in actions:
    for sequence in range(number_of_sequences):
        window = []
        for frame_num in range(every_sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
sequences[0]

[array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.39542124,  0.63193005,
         0.        ,  0.48866868,  0.62292951, -0.0

In [ ]:
X = np.array(sequences)

In [ ]:
X

array([[[ 0.        ,  0.        ,  0.        , ...,  0.35959291,
          0.48585761, -0.0107872 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.34653437,
          0.48195836, -0.00795674],
        [ 0.        ,  0.        ,  0.        , ...,  0.37665778,
          0.484368  , -0.01713235],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.37632918,
          0.50157231,  0.0304199 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.37413582,
          0.49998099,  0.02536973],
        [ 0.        ,  0.        ,  0.        , ...,  0.37121937,
          0.49954426,  0.02845585]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.28854802,
          0.64280248,  0.04286661],
        [ 0.        ,  0.        ,  0.        , ...,  0.28911313,
          0.63620001,  0.04674198],
        [ 0.        ,  0.        ,  0.        , ...,  0.29142988,
          0.59800792,  0.03665687],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [ ]:
X.shape

(2300, 30, 126)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
y.shape

(2300, 23)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
#_____________________________________________________________________________
# X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
# X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
#_____________________________________________________________________________
X_train_and_val, X_test, y_train_and_val, y_test = train_test_split(X,y, train_size=0.9)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_and_val,y_train_and_val, test_size=0.2)

In [ ]:
X_train.shape

(1656, 30, 126)

In [ ]:
X_valid.shape

(414, 30, 126)

In [ ]:
X_test.shape

(230, 30, 126)

In [ ]:
y_train.shape

(1656, 23)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import GRU

In [ ]:
batch_size = 128
epochs = 1000

log_dir = os.path.join('/content/drive/MyDrive/Logs', datetime.datetime.now().strftime("%Y.%m.%d-%H.%M") + '--batch__' + str(batch_size) + '--epochs__' + str(epochs))
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))

# # ______________________________________
# # model = Sequential()
# # model.add(LSTM(64, return_sequences=False, activation='relu', input_shape=(30,126)))
# # model.add(Dense(32, activation='relu'))
# # model.add(Dense(actions.shape[0], activation='softmax'))

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, LSTM, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


# Transformer block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build the Transformer model
def build_transformer_model(input_shape, num_classes, embed_dim, num_heads, ff_dim, num_blocks, lstm_units):
    inputs = Input(shape=input_shape)
    x = Dense(embed_dim)(inputs)

    for _ in range(num_blocks):
      x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

    # Add LSTM layers
    x = LSTM(lstm_units, return_sequences=True)(x)
    x = LSTM(lstm_units)(x)
    x = Dropout(0.1)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model parameters
input_shape = (30, 126)
num_classes = 23
embed_dim = 128
num_heads = 2 #8
ff_dim = 128
num_blocks = 6 #4
lstm_units = 64

# Compile the model
model = build_transformer_model(input_shape, num_classes, embed_dim, num_heads, ff_dim, num_blocks,lstm_units)
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=[CategoricalAccuracy()])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('transformer_sign_language.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

# Evaluate the model
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

Epoch 1/100
52/52 [==============================] - 37s 368ms/step - loss: 3.1494 - categorical_accuracy: 0.0411 - val_loss: 3.1391 - val_categorical_accuracy: 0.0411
Epoch 2/100
52/52 [==============================] - 17s 332ms/step - loss: 3.1424 - categorical_accuracy: 0.0435 - val_loss: 3.1391 - val_categorical_accuracy: 0.0411
Epoch 3/100
52/52 [==============================] - 20s 386ms/step - loss: 3.1380 - categorical_accuracy: 0.0435 - val_loss: 3.1420 - val_categorical_accuracy: 0.0338
Epoch 4/100
52/52 [==============================] - 17s 331ms/step - loss: 3.1377 - categorical_accuracy: 0.0447 - val_loss: 3.1440 - val_categorical_accuracy: 0.0556
Epoch 5/100
52/52 [==============================] - 17s 328ms/step - loss: 3.1381 - categorical_accuracy: 0.0465 - val_loss: 3.1428 - val_categorical_accuracy: 0.0338
Epoch 6/100
52/52 [==============================] - 18s 338ms/step - loss: 3.1383 - categorical_accuracy: 0.0380 - val_loss: 3.1386 - val_categorical_accuracy:

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, LSTM, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# Normalize each sample independently
def normalize_sample(sample, epsilon=1e-8):
    mean = sample.mean(axis=0)
    std = sample.std(axis=0) + epsilon
    return (sample - mean) / std

def normalize_data(X, epsilon=1e-8):
    return np.array([normalize_sample(sample, epsilon) for sample in X])

X_train = normalize_data(X_train)
X_valid = normalize_data(X_valid)
X_test = normalize_data(X_test)

# Check for NaNs in the data
print("Any NaNs in X_train?", np.isnan(X_train).any())
print("Any NaNs in X_valid?", np.isnan(X_valid).any())
print("Any NaNs in X_test?", np.isnan(X_test).any())

# Transformer block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build the hybrid model
def build_hybrid_model(input_shape, num_classes, embed_dim, num_heads, ff_dim, num_transformer_blocks, lstm_units):
    inputs = Input(shape=input_shape)
    x = Dense(embed_dim)(inputs)

    # Add Transformer blocks
    for _ in range(num_transformer_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

    # Add LSTM layers
    x = LSTM(lstm_units, return_sequences=True)(x)
    x = LSTM(lstm_units)(x)

    # Add classification layers
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model parameters
input_shape = (30, 126)
num_classes = 23
embed_dim = 32#128
num_heads = 2#4
ff_dim = 32#128
num_transformer_blocks = 6
lstm_units = 64

# Compile the model
optimizer = Adam(learning_rate=0.0001, clipvalue=1.0)  # Gradient clipping
model = build_hybrid_model(input_shape, num_classes, embed_dim, num_heads, ff_dim, num_transformer_blocks, lstm_units)
model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=[CategoricalAccuracy()])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs')

# Train the model
# history = model.fit(X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_valid, y_valid), callbacks=[early_stopping, tensorboard])
history = model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_data=(X_valid, y_valid), callbacks=[tensorboard])

# Evaluate the model
results = model.evaluate(X_test, y_test, batch_size=16)
print("test loss, test acc:", results)


Any NaNs in X_train? False
Any NaNs in X_valid? False
Any NaNs in X_test? False
Epoch 1/1000
104/104 [==============================] - 33s 113ms/step - loss: 3.7744 - categorical_accuracy: 0.0441 - val_loss: 3.7271 - val_categorical_accuracy: 0.0411
Epoch 2/1000
104/104 [==============================] - 8s 76ms/step - loss: 3.7187 - categorical_accuracy: 0.0507 - val_loss: 3.6781 - val_categorical_accuracy: 0.0749
Epoch 3/1000
104/104 [==============================] - 10s 95ms/step - loss: 3.6769 - categorical_accuracy: 0.0525 - val_loss: 3.6364 - val_categorical_accuracy: 0.0894
Epoch 4/1000
104/104 [==============================] - 10s 94ms/step - loss: 3.6337 - categorical_accuracy: 0.0574 - val_loss: 3.5944 - val_categorical_accuracy: 0.0870
Epoch 5/1000
104/104 [==============================] - 9s 83ms/step - loss: 3.5836 - categorical_accuracy: 0.0737 - val_loss: 3.5516 - val_categorical_accuracy: 0.1159
Epoch 6/1000
104/104 [==============================] - 10s 96ms/step -

In [ ]:
!tensorboard --logdir logs/fit

2024-07-24 15:10:07.580512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 15:10:07.627122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 15:10:07.641461: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 15:10:10.183481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to 

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
!nvidia-smi

In [ ]:
model.fit(X_train, y_train, epochs=epochs, shuffle=True, batch_size=batch_size, validation_data= (X_valid, y_valid), callbacks=[tb_callback])

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, LSTM, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# Normalize each sample independently
def normalize_sample(sample, epsilon=1e-8):
    mean = sample.mean(axis=0)
    std = sample.std(axis=0) + epsilon
    return (sample - mean) / std

def normalize_data(X, epsilon=1e-8):
    return np.array([normalize_sample(sample, epsilon) for sample in X])

X_train = normalize_data(X_train)
X_valid = normalize_data(X_valid)
X_test = normalize_data(X_test)

# Check for NaNs in the data
print("Any NaNs in X_train?", np.isnan(X_train).any())
print("Any NaNs in X_valid?", np.isnan(X_valid).any())
print("Any NaNs in X_test?", np.isnan(X_test).any())

# Transformer block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build the hybrid model
def build_hybrid_model(input_shape, num_classes, embed_dim, num_heads, ff_dim, num_transformer_blocks, lstm_units):
    inputs = Input(shape=input_shape)
    x = Dense(embed_dim)(inputs)

    # Add Transformer blocks
    for _ in range(num_transformer_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

    # Add LSTM layers
    x = LSTM(lstm_units, return_sequences=True)(x)
    x = LSTM(lstm_units)(x)

    # Add classification layers
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model parameters
input_shape = (30, 126)  # Shape of each sample
num_classes = 23
embed_dim = 128
num_heads = 2  # Number of attention heads
ff_dim = 4  # Feed forward network dimension
num_transformer_blocks = 6  # Number of transformer blocks
lstm_units = 64  # Number of LSTM units

# Compile the model
optimizer = Adam(learning_rate=0.0001, clipvalue=1.0)  # Gradient clipping
model = build_hybrid_model(input_shape, num_classes, embed_dim, num_heads, ff_dim, num_transformer_blocks, lstm_units)
model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=[CategoricalAccuracy()])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_valid, y_valid), callbacks=[early_stopping, tensorboard])

# Evaluate the model
results = model.evaluate(X_test, y_test, batch_size=16)
print("test loss, test acc:", results)
